In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from gtfparse import read_gtf
import muon as mu
import scipy.io
import csv
import os

In [ ]:
sc_file = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files'
mudata = mu.read(f'{sc_file}/multi_all_cells_raw.h5mu')

In [ ]:
rna = mudata['rna']
adata.var['seqname'] = adata.var['interval'].str.split(':')[0]

In [ ]:
[x[0] for x in rna.var['interval'].str.split(':')]

In [ ]:
gtf_fn = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/refdata-cellranger-arc-mm10-2020-A-2.0.0/genes/genes.gtf'
rna_data = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/soupx'
sc_file = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files'
h5_file = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/filtered_feature_bc_matrix.h5'
qc = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/figures/qc'
tf_matrix_dir = "/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/analysis/tf_analysis/filtered_tf_bc_matrix"

os.makedirs(sc_file, exist_ok=True)
os.makedirs(qc, exist_ok=True)



var = pd.read_csv(
    '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/filtered_feature_bc_matrix/features.tsv.gz',
    header=None,sep = '\t', index_col=None)
var.columns = ['gene_id','gene_name_feature','measure','seqname','num1','num2']
var.set_index('gene_name_feature',drop=False,inplace=True)
var.rename_axis('feature',inplace=True)
gtf = read_gtf(gtf_fn)
print(gtf.head(1))
adata = sc.read_10x_mtx(rna_data)
adata.var = var[['gene_id','gene_name_feature',]].loc[var['measure']=='Gene Expression']
## Add .var columns from gtf
for column in ['gene_type','gene_name','seqname']:
    temp_dict = pd.Series(gtf[column].values, index=gtf['gene_id']).to_dict()
    adata.var[column] = [temp_dict[x] for x in adata.var['gene_id'].values]
mudata = mu.read_10x_h5("/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/filtered_feature_bc_matrix.h5")
mudata.write(f'{sc_file}/multi_all_cells_raw.h5mu')


mudata['rna'].X = adata.X
mudata['rna'].var = adata.var
mudata['rna'].var_names = adata.var_names
mudata['rna'].obs = adata.obs
mudata['rna'].obs_names = adata.obs_names

# tf-bc matrix=
adata_tf = sc.read_mtx(f"{tf_matrix_dir}/matrix.mtx")

motifs_path = f"{tf_matrix_dir}/motifs.tsv"
var = pd.read_csv(motifs_path, sep='\t', index_col=0, header=None)
barcodes_path = f"{tf_matrix_dir}/barcodes.tsv"
obs = pd.read_csv(barcodes_path, sep='\t', index_col=0, header=None)
adata_tf = adata_tf.T
adata_tf.layers['raw'] = adata_tf.X.copy()
adata_tf.obs_names = obs.index.values
adata_tf.var_names = var.index.values
mudata.mod['tfs'] = adata_tf
del mudata.mod['tfs'].layers['raw']
## Add .obs columns custom
mouse = []
sex= []
for x in adata.obs_names:
    if x.split('-')[1] == '1':
        mouse.append('nor-1')
        sex.append('F')
    elif x.split('-')[1] == '2':
        mouse.append('nor-2')
        sex.append('F')
    elif x.split('-')[1] == '3':
        mouse.append('nor-3')
        sex.append('M')
    elif x.split('-')[1] == '4':
        mouse.append('hyp-1')
        sex.append('F')
    elif x.split('-')[1] == '5':
        mouse.append('hyp-2')
        sex.append('M')
mudata.mod['rna'].obs['mouse'] = mouse
mudata.mod['rna'].obs['sex'] = sex
mudata.mod['rna'].obs['treatment'] = ['Hyperoxia' if x.split('-')[0].startswith('h') else 'Normoxia' for x in adata.obs['mouse']]
mudata.mod['rna'].obs['timepoint'] = 'P7'
mudata.mod['atac'].uns['atac'] = dict(mudata.mod['atac'].uns['atac'])  # anndata does not write ordered dict
mudata.mod['atac'].uns['files'] = dict(mudata.mod['atac'].uns['files'])  # anndata does not write ordered dict
mudata.obs = mudata['rna'].obs.copy()
# mudata.update()
print(mudata)


In [ ]:

mudata.write(f'{sc_file}/multi_all_cells_raw.h5mu')

In [ ]:
mudata.obsm

In [ ]:
adata_rna = sc.read('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/multiome_gex_processed_cell_typed_raw.gz.h5ad')
adata_rna

In [ ]:
muon_adata = mu.read_10x_h5("/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/filtered_feature_bc_matrix.h5")

In [ ]:
del muon_adata.var['interval']

In [ ]:
muon_adata = muon_adata[adata_rna.obs_names,:]
mu_rna = muon_adata['rna'].copy()
mu_rna.var_names_make_unique()
mu_rna = mu_rna[:,adata_rna.var_names]
mu_rna.layers['soupx'] = adata_rna.X.copy()
mu_rna.obs = adata_rna.obs[['mouse', 'sex', 'treatment', 'timepoint','lineage', 'celltype']]


mu_atac = muon_adata['atac'].copy()
mu_atac.obs = adata_rna.obs[['mouse', 'sex', 'treatment', 'timepoint','lineage', 'celltype']]
mu_atac.uns['atac'] = dict(mu_atac.uns['atac']) # anndata does not write ordered dict
mu_atac.uns['files'] = dict(mu_atac.uns['files']) # anndata does not write ordered dict

In [ ]:
mu_rna.obs.to_csv('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_metadata.csv')

In [ ]:
mu_rna.write('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_rna.gz.h5ad',
                       compression='gzip')
mu_atac.write('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_atac.gz.h5ad',
                       compression='gzip')

In [ ]:
mu_atac.uns['files']

In [ ]:
sc.pl.dotplot(adata_rna, ['Cxcl10'], groupby=['celltype','treatment'])

In [ ]:
mu_rna.layers['soupx']

In [ ]:
adata_rna.obs[['mouse', 'sex', 'treatment', 'timepoint','lineage', 'celltype']]

In [ ]:
muon_adata.obs

In [ ]:
muon_adata.var_names

In [ ]:
adata_rna

In [ ]:
adata_rna

In [ ]:
adata_atac = muon_adata['atac']
adata_atac.var_names

In [ ]:
# tf-bc matrix
import scipy.io
import csv
import os 
matrix_dir = "/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/outs/analysis/tf_analysis/filtered_tf_bc_matrix"
adata_tf = sc.read_mtx(os.path.join(matrix_dir, "matrix.mtx"))
 
motifs_path = os.path.join(matrix_dir, "motifs.tsv")
var = pd.read_csv(motifs_path, sep='\t', index_col =0,header=None)
barcodes_path = os.path.join(matrix_dir, "barcodes.tsv")
obs = pd.read_csv(barcodes_path, sep='\t', index_col =0,header=None)
adata_tf = adata_tf.T
adata_tf.layers['raw']= adata_tf.X.copy()
adata_tf.obs_names=obs.index.values
adata_tf.var_names=var.index.values


adata_tf

In [ ]:
import h5py
fl = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/analysis/feature_linkage/feature_linkage_matrix.h5'
f = h5py.File(fl,
              'r')
[key for key in f.keys()]


In [ ]:
for key in f.keys():
    print(key)
    d = f[key]
    try:
        print(type(d))
        print(d.keys())
#         for key2 in d.keys():
#             print(key2)
#             print(d[key2].head())
    except:
        print(type(d))
        print(d.shape)

In [ ]:
score = f['score']
sig = f['significance']
indices = f['indices']
indptr = f['indptr']


In [ ]:
from scipy.sparse import csr_matrix
score_mtx = csr_matrix((score,indices,indptr), shape=(118535,118535))
sig_mtx = csr_matrix((sig,indices,indptr), shape=(118535,118535))

In [ ]:
from anndata import AnnData
score_mtx

In [ ]:
top_5 = np.argpartition(score_mtx,-5,axis = 1)[-5:]


In [ ]:
f['features']['name']

In [ ]:
indptr = f['indptr']
indptr[14]

In [ ]:
sig = f['significance']

In [ ]:
sig[12]

In [ ]:
score = f['score']
score[20]

In [ ]:
import collections
import scipy.sparse as sp_sparse
import tables
 
CountMatrix = collections.namedtuple('CountMatrix', ['feature_ref', 'barcodes', 'matrix'])
 
def get_matrix_from_h5(filename):
    with tables.open_file(filename, 'r') as f:
        mat_group = f.get_node(f.root, 'matrix')
        barcodes = f.get_node(mat_group, 'barcodes').read()
        data = getattr(mat_group, 'data').read()
        indices = getattr(mat_group, 'indices').read()
        indptr = getattr(mat_group, 'indptr').read()
        shape = getattr(mat_group, 'shape').read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)
         
        feature_ref = {}
        feature_group = f.get_node(mat_group, 'features')
        feature_ids = getattr(feature_group, 'id').read()
        feature_names = getattr(feature_group, 'name').read()
        feature_types = getattr(feature_group, 'feature_type').read()
        feature_ref['id'] = feature_ids
        feature_ref['name'] = feature_names
        feature_ref['feature_type'] = feature_types
        tag_keys = [key.decode() for key in getattr(feature_group, '_all_tag_keys').read()]
        for key in tag_keys:
            feature_ref[key] = getattr(feature_group, key).read()
         
        return CountMatrix(feature_ref, barcodes, matrix)
 
filtered_matrix_h5 = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/analysis/feature_linkage/feature_linkage_matrix.h5'
filtered_feature_bc_matrix = get_matrix_from_h5(filtered_matrix_h5)

In [ ]:
tf_bed = pd.read_csv("/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/outs/analysis/tf_analysis/peak_motif_mapping.bed",
                     sep='\t',
                     index_col=None,
                     header=None
                    )


In [ ]:
tf_bed

In [ ]:
adata_atac.uns['atac']['peak_annotation'].loc['Slc6a2']

In [ ]:
var[1].loc[var[1].str.startswith('Sox')]